## **Ridge Regression in Python Notes**

Noah Rubin

May 2021

#### **Main Ideas**

* [Ridge regression](https://machinelearningmastery.com/ridge-regression-with-python/) extends the concepts of OLS but makes some subtle adjustments through [Tikhonov regularisation](http://anderson.ece.gatech.edu/ece6254/assets/11-regression-regularisation.pdf)
* The idea behind ridge regression is to address the concept of the bias-variance tradeoff in machine learning that suggests that optimising one tends to degrade the other
* Ridge regression purposely introduces bias into the regression model in an effort to reduce the variance, which can then potentially lower the mean squared error of our estimator, since $$\text{MSE} = \text{Bias}^2 + \text{Variance}$$
* Even though by the Gauss-Markov theorem, OLS has the lowest sampling variance out of any linear unbiased estimator, there may be a biased estimator that can achieve a lower mean squared error, such as the ridge estimator
* Ridge regression is also a tool to help reduce the impact of multicollinearity within our feature matrix 

---

#### **Algorithm Details**

The loss function for OLS regression is given as:

$$J(\beta_0, \beta_1, ... , \beta_p) = RSS = \sum_{i=1}^{n} (y_i - \beta_0 - \sum_{j=1}^{p}\beta_j x_{i,j})^2.$$

This can be expressed in matrix form as:
$$J(\vec{\beta}) = (\vec{y} - X\vec{\beta})^T(y - X\vec{\beta})$$

---

Ridge regression makes a small modification to the OLS loss function, through adding a shrinkage penalty through [L2 regularisation penalty](https://towardsdatascience.com/intuitions-on-l1-and-l2-regularisation-235f2db4c261#f810), hence for ridge regression:

$$J(\beta_0, \beta_1, ... , \beta_p) = \sum_{i=1}^{n} (y_i - \beta_0 - \sum_{j=1}^{p}\beta_j X_{i,j})^2 + \lambda \sum_{j=1}^{p} \beta_j ^2$$

This can be expressed in matrix form as:

$$J(\vec{\beta}) = (\vec{y} - X\vec{\beta})^T(y - X\vec{\beta}) + \lambda\vec{\beta}^{T}\vec{\beta}$$

<b>By convention, columns in $X$ are assumed to have zero mean and unit variance (after scaling), and the response vector $\vec{y}$ is sometimes centered to have mean zero.<b>

---

The lambda parameter $\lambda \in [0, \infty)$ is a constant that can be chosen through resampling methods such as cross validation. Ultimately, if $\lambda = 0$ in the final model, the shrinkage penalty (the second term) disappears and we get OLS coefficient estimates. As $\lambda$ gets larger, the shrinkage penalty becomes increasingly pertinent, and coefficient estimates will tend towards zero (but will not be exactly zero). Since $\lambda$ is a hyperparameter that can be tuned, we get different coefficient estimates depending on which value for $\lambda$ is chosen. Ultimately the shrinkage penalty aims to encourage simpler models that have smaller values for the coefficients as "it turns out that shrinking the coefficient estimates can significantly reduce their variance" - *An Introduction to Statistical Learning: With Applications in R*.
    
Also, the size constraint on the coefficients in the ridge
regression "alleviates the problem of large coefficients (in absolute value) and its high variance, which may be a consequence of multicollinearity." - *Rice University STAT 410 Lecture Slides*

[Resource linked here](https://cpb-us-e1.wpmucdn.com/blogs.rice.edu/dist/e/8375/files/2017/08/Lecture16-1l5v69b.pdf) 

--- 

Expanding the terms in the loss function, we get

$$J(\vec{\beta}) = \vec{y}^T\vec{y} -2\vec{\beta}^TX^T \vec{y} + \beta^TX^TX\vec{\beta} + \lambda\vec{\beta}^{T}\vec{\beta}$$

which is a convex function with a closed form solution when optimising coefficients. Taking the derivative of the loss function with respect to the beta vector we obtain:

$$\frac{\partial J(\vec{\beta})}{\partial \vec{\beta}} = -2X^{T}\vec{y} + 2X^{T}X\vec{\beta} + 2\lambda\vec{\beta}$$

Since $J(\vec{\beta})$ is convex, to minimise this quantity, we can set the derivative equal to 0 to find an estimate $\vec{b}_{ridge}$ for $\vec{\beta}$ thus:

$$-2X^{T}\vec{y} + 2X^{T}X\vec{b} + 2\lambda\vec{b} = 0$$

Moving, $-2X^{T}\vec{y}$ to the other side, and dividing terms by two, we get 

$$X^{T}X\vec{b} + \lambda\vec{b} = X^{T}\vec{y}$$

Factorising out a common factor of $\vec{b}$ we get

$$(X^{T}X + \lambda I)\vec{b} = X^{T}\vec{y}$$

"Pre-multiplying" both sides by $(X^{T}X + \lambda I)^{-1}$ allows us to obtain

$$\vec{b}_{ridge} = (X^{T}X + \lambda I)^{-1}X^{T}\vec{y}$$

Including a positive lambda ensures that we obtain a non singular matrix for $(X^{T}X + \lambda I)^{-1}$, even if $X^TX$ is singular (not of full rank)

This optimisation problem to find $\vec{b}_{ridge}$ could have also been solved using [Lagrange Multipliers](https://en.wikipedia.org/wiki/Lagrange_multiplier), where we would find our estimator using the Karush Kuhn-Tucker (KKT) multiplier method.

$$\text{argmin}_{||\vec{\beta}||_2 ^2 \leq c}||\vec{y} - X\vec{\beta}||_2 ^2$$

where we optimise the beta vector subject to the constraint that $\sum_{j=1}^p \beta_{j}^2 \leq c$.



---

**Proving that $\vec{b}_{ridge}$ is biased:**

From above,

$$\vec{b}_{ridge} = (X^{T}X + \lambda I)^{-1}X^{T}\vec{y}$$

Let $M = X^{T}X$, then:

$$\vec{b}_{ridge} = (M + \lambda I)^{-1}M(M^{-1}X^{T}\vec{y})$$

Factorising $M$ out in the first term and substituting the expression for $M$ into the second term, we obtain:

$$\vec{b}_{ridge} = [M(I + \lambda M^{-1})]^{-1}M[(X^TX)^{-1}X^T\vec{y}]$$

Since by matrix inverse laws, $(AB)^{-1} = B^{-1}A^{-1}$, and since $\vec{b}_{ols} = (X^TX)^{-1}X^T\vec{y}$:

$$\vec{b}_{ridge} = (I + \lambda M^{-1})^{-1}M^{-1}M\vec{b}_{ols}$$

Since $A^{-1}A$ is the identity matrix for a matrix $A$, then:

$$\vec{b}_{ridge} = (I + \lambda M^{-1})\vec{b}_{ols}$$

Taking the expectation of this simplified quantity, 

$$E(\vec{b}_{ridge}) = E((I + \lambda M^{-1})\vec{b}_{ols})$$

As $(I + \lambda M^{-1})$ is not random and as the OLS estimator under Gauss Markov assumptions is unbiased, 

$$E(\vec{b}_{ridge}) = (I + \lambda M^{-1})\vec{\beta}_{ols}$$

Which is not equal to $\vec{\beta}_{ols}$ if lambda is non-zero (and positive). But if lambda was zero then it is technically not ridge regression but rather just OLS.

---

**Variance of the ridge estimator**

The variance of the OLS estimator was shown in a previous jupyter notebook to be given as:

$$\text{Var}(\vec{b}_{ols}) = \sigma^2(X^TX)^{-1}$$

The ridge estimator of $\vec{\beta}$ can be given as 
$$\vec{b}_{ridge} = (X^{T}X + \lambda I)^{-1}X^{T}\vec{y}$$

This can also be expressed as,

$$\vec{b}_{ridge} = (X^{T}X + \lambda I)^{-1}X^{T}X(X^{T}X)^{-1}X^T\vec{y}$$

Since $(X^{T}X)^{-1}X^T\vec{y} = \vec{b}_{ols}$,

$$\vec{b}_{ridge} = (X^{T}X + \lambda I)^{-1}X^{T}X\vec{b}_{ols}$$

Taking the variance of both sides:

$$\text{Var}(\vec{b}_{ridge}) = \text{Var}((X^{T}X + \lambda I)^{-1}X^{T}X\vec{b}_{ols})$$

As $\vec{b}_{ols}$ is a random vector, 

$$\text{Var}(\vec{b}_{ridge}) = (X^{T}X + \lambda I)^{-1}X^{T}X\text{Var}(\vec{b}_{ols})((X^{T}X + \lambda I)^{-1}X^{T}X)^T$$

Recognising that $\text{Var}(\vec{b}_{ols}) = \sigma^2(X^TX)^{-1}$ under the homoskedasticity assumption, and by applying the idea that $(AB)^T = B^TA^T$ for matrices $A$ and $B$

$$\text{Var}(\vec{b}_{ridge}) = (X^{T}X + \lambda I)^{-1}X^{T}X\sigma^2 (X^TX)^{-1}X^{T}X(X^{T}X + \lambda I)^{-1}$$

Cancelling terms out and assuming $\sigma^2$ is constant,

$$\text{Var}(\vec{b}_{ridge}) = \sigma^2(X^{T}X + \lambda I)^{-1}X^{T}X(X^{T}X + \lambda I)^{-1}$$
    
The variance of the ridge estimator is always lower than that of OLS. The [proof](https://www.statlect.com/fundamentals-of-statistics/ridge-regression ) is quite long so consider a case where
$X^TX = I$
    
If we substitute $X^TX = I$ into the equation above, we obtain
    
$$\text{Var}(\vec{b}_{ridge}) = \sigma^2(I + \lambda)^{-1}(I + \lambda)^{-1}$$

Factorising out the identity matrix
    
$$\text{Var}(\vec{b}_{ridge}) = \sigma^2(1 + \lambda)^{-1}(1 + \lambda)^{-1}I$$

Simplifying, we get

$$\text{Var}(\vec{b}_{ridge}) = \sigma^2(1 + \lambda)^{-2}I$$
    
Which is certainly lower than the variance of the OLS estimator. Ultimately, different values of lambda will allow us to control both the magnitiude of the variance and the coefficients


---


**Useful property of the ridge estimator**

In cases whether the columns of $X$ are orthonormal (i.e. the columns are orthogonal and each have unit length), then this orthogonal matrix $X$ adheres to:
$$X^TX = X^{-1}X = I.$$ 

More profoundly, if can also be shown that when this condition is met, the ridge estimator is a multiple of the OLS estimator such that,

$$\vec{b}_{ridge} = \frac{1}{1 + \lambda}\vec{b}_{ols}$$
    
If we were now to take the expectation of this quantity, we'd see that ridge estimator, on average, underestimates the true coefficient since 
$$E(\vec{b}_{ridge}) = \frac{1}{1+\lambda}E(\vec{b}_{ols}) = \frac{1}{1+\lambda}\beta$$



Extra resource [here](https://arxiv.org/pdf/1509.09169.pdf)









In [1]:
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge

# Personal display settings
#===========================

# Suppress scientific notation
np.set_printoptions(suppress=True)

# Get dataset values showing only 2dp
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_colwidth', None)

# For clear plots with a nice background
plt.style.use('seaborn-whitegrid') 
%matplotlib inline

%load_ext autoreload
%autoreload 2

# python files
import data_prep
import helper_funcs

In [2]:
train = pd.read_csv('../datasets/train_updated.csv')
test = pd.read_csv('../datasets/test_updated.csv')

In [3]:
# Split data
to_drop = ['Country', 'HDI', 'Life_exp']

X_train = train.drop(to_drop, axis='columns')
X_test = test.drop(to_drop, axis='columns')

y_train = train['Life_exp']
y_test = test['Life_exp']

In [4]:
pipe = data_prep.create_pipeline(Ridge())
pipe

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numeric',
                                                  Pipeline(steps=[('ss',
                                                                   StandardScaler())]),
                                                  ['GDP_cap']),
                                                 ('categorical',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['Status'])])),
                ('imputation', KNNImputer()), ('model', Ridge())])

In [5]:
param_grid = {
    'imputation__n_neighbors': np.arange(3, 21, 2), 
    'imputation__weights': ['uniform', 'distance'],
    'model__alpha': np.linspace(0, 3, 15)  # sklearn calls it alpha instead of lambda
}

best_estimator, best_params = data_prep.exhaustive_search(X_train, y_train, pipe, param_grid, cv=10, scoring='neg_mean_squared_error')
final_model = best_estimator.fit(X_train, y_train)
print(f"Best parameters: {best_params}")

Best parameters: {'imputation__n_neighbors': 3, 'imputation__weights': 'distance', 'model__alpha': 3.0}


### Evaluate model on the test set

In [6]:
r2, mse, rmse, mae = helper_funcs.display_regression_metrics(y_test, final_model.predict(X_test))

R^2 = 0.9357379493482635
Mean Squared Error = 5.206254655759958
Root Mean Squared Error = 2.281721862050666
Mean Absolute Error = 1.7668402548997668


### Save piepline for future use

In [7]:
joblib.dump(final_model, './saved_models/Ridge Regression.joblib')

['./saved_models/Ridge Regression.joblib']

### Make a prediction

- Year 2038
- Infant Mortality of 2.91
- 32.56% of GDP is spent on health in 2038
- GDP per capita is 89,570
- Employment to population ration (age 15+) is 64.49%
- Developed Country
- Average years of schooling is 11.79
- 100% of the population has access to electricity

In [8]:
saved_pipeline = joblib.load('./saved_models/Ridge Regression.joblib')
input_data = [2038, 2.91, 32.56, 89570, 64.49, 'Developed', 11.79, 100]

print(f"Predicted life expectancy (using ridge regression) = {helper_funcs.make_prediction(input_data, saved_pipeline, X_test)}")

Predicted life expectancy (using ridge regression) = 90.96883045598514
